In [ ]:
# Import necessary libraries
import os
from vllm import LLM, SamplingParams
from vllm.steer_vectors.request import SteerVectorRequest
import json

# Set environment variables
os.environ["VLLM_USE_V1"] = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
# Initialize LLM
llm = LLM(
    model="/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/",
    enable_steer_vector=True,
    enforce_eager=True,
    tensor_parallel_size=1
)

# Define math problems for testing
file_path = "/home/xhl/eval/my_eval/data/math/train.jsonl"
problems = []
with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        item = json.loads(line)
        problems.append(item["problem"])

# Create prompt texts from problems
texts = ["Please reason step by step, and put your final answer within \\boxed{}.\nUser: " + problem + "\nAssistant: <think>" for problem in problems]

# Generate answers using the LLM
answers = llm.generate(
    texts[:1000],
    SamplingParams(
        temperature=0,
        max_tokens=8192,
        skip_special_tokens=False,
    ),
)
answers = [answer.outputs[0].text for answer in answers]

In [ ]:
import json

# 假设 texts 和 answers 已经生成
# 保存到文件
save_path = "results.json"
with open(save_path, "w", encoding="utf-8") as f:
    json.dump({"texts": texts[:1000], "answers": answers}, f, ensure_ascii=False, indent=2)

print(f"已保存到 {save_path}")


In [ ]:
with open("results.json", "r", encoding="utf-8") as f:
    data = json.load(f)
    texts = data["texts"]
    answers = data["answers"]

print("加载成功，数量：", len(texts), len(answers))

In [ ]:
from transformers import AutoTokenizer

# Create QA pairs by combining prompts and answers
qa_pairs = [texts[i] + answers[i] for i in range(len(texts))]

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/")

# The newline token suffix in tokenizer vocabulary
target_suffix = "ĊĊ"  # "\n\n" is tokenized as "ĊĊ"

# Process each QA pair to find newline positions
all_tokens_list = []
all_newline_positions = []
for qa in qa_pairs:
    # Tokenize the QA pair
    tokens = tokenizer.tokenize(qa, add_special_tokens=True)
    all_tokens_list.append(tokens)
    
    # Find all positions of "ĊĊ" in the tokens
    # These represent potential paragraph breaks in the text
    positions = [
        i for i, token in enumerate(tokens) 
        if isinstance(token, str) and token.endswith(target_suffix)
    ]
    all_newline_positions.append(positions)


In [ ]:
# Define keyword sets for classifying reasoning segments
TRANSITION_KEYWORDS = [
    'alternatively', 'think differently', 'another way', 'another approach',
    'another method', 'another solution', 'another strategy', 'another technique'
]

REFLECTION_KEYWORDS = [
    'wait', 'verify', 'make sure', 'hold on', 'think again', "'s correct",
    "'s incorrect", 'let me check', 'seems right'
]

def classify_segment(text_segment):
    """
    Classify text segments based on keyword matches.
    
    Args:
        text_segment: String of text to classify
        
    Returns:
        String category: "Transition", "Reflection", or "Execution"
    """
    lower_text = text_segment.lower()
    
    if any(keyword in lower_text for keyword in TRANSITION_KEYWORDS):
        return "Transition"
    
    if any(keyword in lower_text for keyword in REFLECTION_KEYWORDS):
        return "Reflection"
    
    # Default category is "Execution" (not "Other")
    return "Execution"

# Perform classification on all QA pairs
all_classifications = []

for i, positions in enumerate(all_newline_positions):
    tokens = all_tokens_list[i]
    classifications_for_qa = []

    # Skip if no paragraph breaks were found
    if not positions:
        all_classifications.append(classifications_for_qa)
        continue

    # Classify each paragraph segment
    for j, pos in enumerate(positions):
        # Define segment boundaries
        start_slice = pos + 1
        end_slice = positions[j+1] if j + 1 < len(positions) else len(tokens)

        # Extract and decode the text segment
        token_slice = tokens[start_slice:end_slice]
        text_segment = tokenizer.decode(
            tokenizer.convert_tokens_to_ids(token_slice), 
            skip_special_tokens=True
        ).strip()
        
        # Classify the segment
        category = classify_segment(text_segment)

        # Store the classification result
        classifications_for_qa.append({
            "position_in_tokens": pos,
            "category": category,
        })

    all_classifications.append(classifications_for_qa)

# Print summary of classification results
print("--- Summary of classification results for all samples ---")

for i, qa_results in enumerate(all_classifications):
    print(f"\n--- Analysis of QA Pair {i+1} ---")

    # Group token positions by category
    summary = {
        "Transition": [],
        "Reflection": [],
        "Execution": []
    }

    # Collect positions for each category
    for result in qa_results:
        category = result["category"]
        position = result["position_in_tokens"]
        if category in summary:
            summary[category].append(position)

    # Print formatted summary by category
    print(f"Transition positions: {summary['Transition']}")
    print(f"Reflection positions: {summary['Reflection']}")
    print(f"Execution positions: {summary['Execution']}")

print("\n" + "="*40)

In [ ]:
import easysteer.hidden_states as hs
from easysteer.steer import StatisticalControlVector
import numpy as np

#-------------------------------------------------
# 1. 初始化 LLM
#-------------------------------------------------
llm = LLM(
    model="/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/",
    task="reward",  # reward 模式才能拿 hidden states
    tensor_parallel_size=1
)

#-------------------------------------------------
# 2. 初始化累加器 (按类别和层数)
#-------------------------------------------------
categories = ["Transition", "Reflection", "Execution"]
sums = {cat: {} for cat in categories}
counts = {cat: {} for cat in categories}
num_layers = None

#-------------------------------------------------
# 3. 流式处理每个样本
#    - 不再一次性保存 all_hidden_states
#    - 逐个样本提取 hidden states
#    - 在线更新均值
#-------------------------------------------------
for sample_idx, qa_results in enumerate(all_classifications):
    # 获取当前样本 hidden states
    hidden_states, outputs = hs.get_all_hidden_states(llm, [qa_pairs[sample_idx]])
    # 确定层数（只需第一次）
    if num_layers is None:
        num_layers = len(hidden_states[0])

    # 遍历当前样本中标注的类别位置
    for result in qa_results:
        category = result["category"]
        pos = result["position_in_tokens"]

        for layer_idx in range(num_layers):
            # 提取该 token 的 hidden state
            try:
                token_hidden = hidden_states[0][layer_idx][pos].cpu().float().numpy()
    
                # 初始化累加器
                if layer_idx not in sums[category]:
                    sums[category][layer_idx] = np.zeros_like(token_hidden)
                    counts[category][layer_idx] = 0
    
                # 累加
                sums[category][layer_idx] += token_hidden
                counts[category][layer_idx] += 1
            except:
                pass

#-------------------------------------------------
# 4. 计算各类别平均向量
#-------------------------------------------------
average_vectors = {
    cat: {
        layer: sums[cat][layer] / counts[cat][layer]
        for layer in sums[cat] if counts[cat][layer] > 0
    }
    for cat in sums
}

#-------------------------------------------------
# 5. 导出 GGUF 文件
#-------------------------------------------------
try:
    model_type_str = llm.config.model_type
except (AttributeError, NameError):
    print("Warning: Could not determine model_type from `llm` object. Using a placeholder.")
    model_type_str = "qwen2"

for category, directions in average_vectors.items():
    if not directions:
        print(f"Skipping '{category}' (no data)")
        continue

    metadata = {
        "source": "Averaged from classified tokens (streaming)",
        "num_vectors_averaged": sum(counts[category].values())
    }

    control_vector = StatisticalControlVector(
        model_type=model_type_str,
        method="Average",
        directions=directions,
        metadata=metadata
    )

    out_path = f"{category.lower()}_avg_vector.gguf"
    control_vector.export_gguf(out_path)
    print(f"Exported {out_path} with {metadata['num_vectors_averaged']} vectors.")


In [ ]:
# Import hidden states module to extract model activations
import easysteer.hidden_states as hs

# Create a new LLM instance in reward mode
# Note: This allows us to extract hidden states rather than generating text
llm = LLM(
    model="/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/",
    task="reward",  # Use reward task to get hidden states
    tensor_parallel_size=1
)

# Extract hidden states for all tokens in the QA pairs
all_hidden_states, outputs = hs.get_all_hidden_states(llm, qa_pairs)

In [ ]:
from easysteer.steer import StatisticalControlVector
import numpy as np

# Step 1: Collect all relevant hidden states by category
#-------------------------------------------------

# Initialize a dictionary to collect all hidden states by category and layer
collected_states = {
    "Transition": {},
    "Reflection": {},
    "Execution": {}
}

# Get the number of layers in the model
num_layers = len(all_hidden_states[0])

# Process each sample's classification results to collect hidden states
for sample_idx, qa_results in enumerate(all_classifications):
    for result in qa_results:
        category = result["category"]
        position = result["position_in_tokens"]

        # For each layer, collect hidden states for tokens of this category
        for layer_idx in range(num_layers):
            # Initialize empty list for this layer if not already present
            if layer_idx not in collected_states[category]:
                collected_states[category][layer_idx] = []
            
            # Extract hidden state from the model output
            token_hidden = all_hidden_states[sample_idx][layer_idx][position]
            
            # Convert to numpy for easier processing
            token_hidden = token_hidden.cpu().float().numpy()
            
            # Store the hidden state
            collected_states[category][layer_idx].append(token_hidden)


# Step 2: Calculate the average hidden state for each category at each layer
#-------------------------------------------------

# Initialize result dictionaries
average_vectors = {
    "Transition": {},
    "Reflection": {},
    "Execution": {}
}

# Track vector counts for metadata
vector_counts = {} 

# Process each category
for category, layer_data in collected_states.items():
    # Skip empty categories
    if not layer_data:
        print(f"Warning: No vectors found for category '{category}'. Skipping.")
        continue

    # Record how many vectors we're averaging for this category
    vector_counts[category] = len(layer_data.get(0, []))
    print(f"Calculating average for '{category}' using {vector_counts[category]} vectors.")

    # Calculate average for each layer
    for layer_idx, vectors in layer_data.items():
        # Calculate mean across all vectors for this layer
        mean_vector = np.mean(np.array(vectors), axis=0)
        average_vectors[category][layer_idx] = mean_vector


# Step 3: Package and export as GGUF files
#-------------------------------------------------

# Try to get model type or use a placeholder
try:
    model_type_str = llm.config.model_type
except (AttributeError, NameError):
    print("Warning: Could not determine model_type from `llm` object. Using a placeholder.")
    model_type_str = "qwen2"  # Default placeholder - adjust as needed for your model

# Create and export control vectors for each category
for category, directions in average_vectors.items():
    if not directions:
        continue  # Skip if no data available
    
    # Prepare metadata for the vector
    metadata = {
        "source": "Averaged from classified newline tokens",
        "num_vectors_averaged": vector_counts.get(category, 0)
    }

    # Create the control vector object
    control_vector = StatisticalControlVector(
        model_type=model_type_str,
        method="Average",
        directions=directions,
        metadata=metadata
    )

    # Export to GGUF format
    control_vector.export_gguf(f"{category.lower()}_avg_vector.gguf")